# EE315 Design Project

Create a Bingo card!

The below method generates an array of random numbers and sorts them into a bingo card format.

Importing the overlay class.

In [1]:
from pynq import Overlay

Program the Overlay onto the PYNQ-Z2, and parse the Hwh file

In [2]:
ol = Overlay("design_1_wrapper.bit")

Return a file that reports the results of the Hwh parsing to the designer.

In [3]:
ol?

Importing the required libraries.

In [4]:
import random
import ipywidgets as widgets
from IPython.display import display
from IPython.display import Audio

In [5]:
rng = ol.rng_test01_0

In [6]:
rng.write(0x00, 0)
rng.write(0x04, 1)
rng.write(0x08,3)
out_address = 0x0C

In [7]:
my_list=[]
for i in range (1,91):
    my_list.append(i)

The call_number_func() ensures that the number generated cannot be greater than 90. If it is then it regenerates it and appends it to a list of random numbers. 

In [8]:
def call_number_func():

    RNG_number = rng.read(out_address)
    while RNG_number>len(my_list):
        RNG_number = rng.read(out_address)
        if RNG_number> len(my_list):
            RNG_number = 90 - len(my_list)

    call_number = my_list[RNG_number]
    my_list.remove(call_number)

    call_list.append(call_number)

In [9]:
call_list=[]

In [88]:
call_number_func()

In [89]:
list(map(int, call_list))

[1, 83]

The function implements the ability for the button to change colour. This will allow the user to interact and score off their numbers which have been called. 

In [35]:
button_list=[]
numPressed=[]
wrongNumberList=[]

In [108]:
def button_on_click(myButton):
    myButton
    myButton.style.button_color = 'green'
    #numPressed = card.children[idx]
    temp = numPressed.find(myButton.description,0,len(numPressed))
    buttons_pressed[myCards].append(temp)

In [91]:
#def bingo(validation):
#    if(validation):
#        print("Congratulations, you have won!")
#    else:
#        print("Unfortunately, your card is incorrect")     

In [92]:
#numMarked = []
#def numberCalledList(numCalled):
    #for noCards in myCards:
        #n = 1
        #numMarked.append(numCalled)
        #n = n + 1

The compareValues function is called when the user presses 'Bingo!'. This function then takes in the numbers pressed by the user on the card and also the numbers called out which have been generated. The function compares values to check that numPressed is a subset of call_list, otherwise it tells the user they are incorrect and shows which values were wrong. 

In [93]:
wrongNumberList = []
def compareValues(numPressed, call_list):
    
    list1=list(map(int, numPressed)) #This changes list from strings to ints
    #list2=list(map(int, call_list))
    list2 = list(dict.fromkeys(call_list)) #converts to dict then back to list to remove duplicates
    numReq = 1 #cardSize*cardSize #number of numbers required for bingo
    
    #check = all(item in list2 for item in list1)
    
    for element in list1:
        if element not in list2:
            wrongNumberList.append(element)
    
    if len(list1)==numReq:
        if (all(item in list2 for item in list1)): 
            print("Congratulations, it's Bingo!")
            display(Audio("./Sounds/ThatsABingo.mp3", autoplay =True))
        else:
            print("Unfortunately, these numbers do not match. These numbers are incorrect " + str(wrongNumberList)) 

In [94]:
#compareValues(call_list, numCalled)

In [95]:
def bingoPressed(bingoButton):
    bingoButton
    bingoButton.style.button_color = 'green'
    bingoButton = compareValues(numPressed, call_list) #

Implementing the bingo card based upon the number of players and size of the card required. 

Through the use of a for loop, the numbers within the card can be generated and placed in a ipywidget layout. This layout creates a grid which is sized using the cardSize * cardSize. 

For the bingo game to be interactive, the need to implement a form of user interaction, such as scoring off the number called, was required. This is achieved by creating a for loop which has a callback element and calls the function above, button_on_click().

In [96]:
cardSize = 4
minNum = 1
maxNum = 90
noCards = 2
myCards = []
bingos = []

for i in range(noCards):
    card = []
    randRange = range(minNum, maxNum)
    card = random.sample(randRange, cardSize * cardSize)
 
    buttons = []
    
    myButton = widgets.GridBox(children=[widgets.Button(description=str(num), layout=widgets.Layout(width='auto', height='auto'),
                 button_style='warning') for num in card],

                layout=widgets.Layout(
                                    width='80%',
                                    grid_template_columns='20%'*cardSize,
                                    grid_template_rows='40px'*cardSize,
                                    grid_gap='1px 2px')
                           )
    
    for button in myButton.children:
        button.on_click(button_on_click)
            
    myCards.append(myButton)
    
  
    
    #bingoButton = widgets.GridBox(children=[widgets.Button(description='Bingo!', layout=widgets.Layout(width='auto', height='auto'),
                 #button_style='warning') for i in range(1)],

                #layout=widgets.Layout(
                                    #width='60%',
                                    #grid_template_columns='20%',
                                    #grid_template_rows='40px',
                                    #grid_gap='1px 2px')
                            
    
    bingoButton = widgets.Button(description='Bingo!', button_style='warning')
    
    #for bingo in bingoButton.children:
    bingoButton.on_click(bingoPressed)

    bingos.append(bingoButton)
    

The bingo card can then be displayed through calling myButton.

In [128]:
numPressed = []
rows, cols = noCards, (cardSize*cardSize)

In [129]:
find = myCards[0].parent 
find = 63

The for loop below allows the bingo cards to be generated and is where the user can interact with the game. 

In [132]:
#numPressed = []
for idx, card in enumerate(myCards):
    #numPressed = myCards[idx]
    display(card)
    for i in range(rows):
        cols = []
        for j in cols:
            cols.append(myButton.description)
    display(bingos[idx])
#numPressed[idx][element] = myCards[idx].children[]    

GridBox(children=(Button(button_style='warning', description='43', layout=Layout(height='auto', width='auto'),…

Button(button_style='warning', description='Bingo!', style=ButtonStyle(button_color='green'))

GridBox(children=(Button(button_style='warning', description='46', layout=Layout(height='auto', width='auto'),…

Button(button_style='warning', description='Bingo!', style=ButtonStyle())

AttributeError: 'Button' object has no attribute 'myButton'

AttributeError: 'Button' object has no attribute 'myButton'

In [47]:
numPressed

[]

In [33]:
call_list

[1]

Prints a list of numbers - keep for now just incase. 

buttons = []
#for num in card:
    #print(num)
for num in card:
    print(num)
    myButton = widgets.Button(description=str(num), layout=widgets.Layout(width='100px', height='100px', columns='20%'*cardSize, rows='40px'*cardSize),
                         button_style='danger')

    buttons.append(myButton)
    myButton.on_click(button_on_click)

    
    

for button in buttons:
    display(button)